In [6]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import xgboost as xgb

from sklearn.cross_validation import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

SEED = 42

In [7]:
''' Performs cross validation procedure 10 times, holding out 20% of training data as the CV set. '''
def cross_validate(x, y, model):
    mean_err = 0.
    for k in range(10):
        # Split the training data into training and testing sets for cross-validation.
        x_train, x_cv, y_train, y_cv = train_test_split(x, y, test_size=0.2, random_state=k*SEED)

        model.fit(x_train, y_train)

        # Calculate training error.
        preds = model.predict(x_cv)
        fold_err = model.score(x_cv, y_cv)
        mean_err += fold_err
        print fold_err
    mean_err /= 10
    return mean_err

In [8]:
''' Fills all missing values in the data with the mean of the respective feature. '''
def fillEmptys(data_frame):
    count = 0
    for feature in data_frame:
        for x in range(len(data_frame[feature])):
            if pd.isnull(data_frame[feature][x]):
                data_frame[feature][x] = data_frame[feature].mean()
                count += 1

In [9]:
def plot_feature(id, feature, title):
    plt.scatter(id, feature)
    plt.title(title)
    plt.show()

In [17]:
train = pd.read_csv('/Users/anthonydepalatis/Documents/Workspace/school/MidtermProject379K/data/train.csv')
test = pd.read_csv('/Users/anthonydepalatis/Documents/Workspace/school/MidtermProject379K/data/test.csv')

#for feature in train:
#    if feature != 'id':
#        plot_feature(train['id'], train[feature], feature)

#fillEmptys(train)
#fillEmptys(test)

# PCA...?
#U, svd, Vt = sp.linalg.svd(train.drop(['id'], axis=1))

gbm = xgb.XGBClassifier()
cross_validate(train.drop(['id', 'Y'], axis=1), train['Y'], gbm)
#gbm = xgb.XGBClassifier(max_depth=10, n_estimators=1000, learning_rate=0.15, objective='binary:logistic').fit(x_train, y_train)

# Make final predictions
gbm.fit(train.drop(['id', 'Y'], axis=1), train['Y'])
test_preds = gbm.predict(test.drop(['id'], axis=1))
submission = pd.DataFrame({"Id":test.id, "Y":test_preds})
submission.to_csv('/Users/anthonydepalatis/Documents/Workspace/school/MidtermProject379K/submission.csv', index=False)

0.9361
0.9368
0.9385
0.9357
0.9396
0.9358
0.933
0.9347
0.93
0.9371
